# Monitors

A monitor is an object that only one thread can execute methods of this object at a time.

In Java Threads there is a `synchronized` modifier making a method a monitor like mutually exclusive.

In python you can use mutexes in the class to achieve that.

In [1]:
from threading import Lock,Thread,Condition,RLock,Semaphore

class Monitor:
    def __init__(self):
        self.mutex = RLock()
        self.count = 0
    
    def increment(self):
        with self.mutex:
            self.count += 1
    
    def get(self):
        with self.mutex:
            return self.count

        
        

In [5]:
def f(mon):
    for i in range(10000):
        mon.increment()

m = Monitor()

t1 = Thread(target=f, args=(m,))
t2 = Thread(target=f, args=(m,))
t3 = Thread(target=f, args=(m,))
t4 = Thread(target=f, args=(m,))

t1.start()
t2.start()
t3.start()
t4.start()
t1.join()
t2.join()
t3.join()
t4.join()
print(m.get())


40000


In [1]:
class Queue:
    def __init__(self,capacity):
        self.capacity = capacity
        self.queue = []
        self.mutex = RLock()
        self.notempty = Lock(0)
        
    def empty(self):
        return len(self.queue) == 0
    
    def full(self):
        return len(self.queue) == self.capacity
    
    def enqueue(self,val):
        with self.mutex:
            while self.full():
                time.sleep(1)
            self.queue.append(val)
            if len(self.queue) == 1:
                self.notempty.release()
            
    def dequeue(self):
        with self.mutex:
            while self.empty():
                self.mutex.release()
                self.notempty.acquire()
                self.mutex.acquire()
            a=self.queue[0]
            del self.queue[0]

# Producer Consumer 

* A queue which is accessed by two (or more) threads. One end a producer thread inserts items, the other end, consumer thread removes and processes the items.
* They work in an infinite loop.
* If queue is empty or full?
* In full and empty cases, they need to check it until queue has an empty slot or has an item respectively. Polling in an infinite loop wastes too much CPU
* Busy waiting is not a good idea:
```python
   while queue.empty():
        time.sleep(1)  # response time will be slow
        pass
```
* Use synchronization methods semaphores or similar to make other end know that queue is ready (not full or not empty)

# Condition Variables

* In a monitor, condition variables let threads to signal each other while keeping the monitor semantics (only one thread inside).
 
```python
c = Condition(mutex)
c.wait()
```
```wait``` does:
```python
c.mutex.release()
# block on condition
# when unblocked:
c.mutex.acquire()
```

Typical usage:
```python
c.acquire() # or acquire mutex of c on construction
.....
while actual condition:
     c.wait()
```

The notifier cannot guarantee that the condition holds semantically and notified thread can directly assume condition holds.

`c.notify()` will unblock one of the threads blocking on condition.

`c.notifyAll()` will unblock all of them. However they still wait on the mutex after unblocking. They enter monitor one at a time.
asdasd




In [7]:
import random
import time

class PCQueue:
    def __init__(self, capacity=10):
        self.mutex=RLock()
        self.queue = []
        self.capacity = capacity
        self.notempty = Condition(self.mutex)
        self.notfull = Condition(self.mutex)
    def empty(self):
        with self.mutex:
            return len(self.queue) == 0
    def full(self):
        with self.mutex:
            return len(self.queue) == self.capacity
    def enqueue(self,item):
        with self.mutex:
            while len(self.queue) == self.capacity:
                print("queue is full, waiting")
                self.notfull.wait()
            
            self.queue.append(item)
            self.notempty.notify()
            
    def dequeue(self):
        with self.mutex:
            while len(self.queue) == 0:
                print("queue is empty, waiting")
                self.notempty.wait()
                
            val = self.queue[0]
            del self.queue[0]
            self.notfull.notify()
            return val
            

def producer(pcq):
    for i in range(30):
        time.sleep(0.15+random.random()*0.15)
        pcq.enqueue(random.randint(0,100))
        print("enqueued")
    print("producer finished")
        
def consumer(pcq):
    for i in range(30):
        time.sleep(0.35+random.random()*0.2)
        print("dequeued ",pcq.dequeue())
    print("consumer finished")
        
q = PCQueue()

prod = Thread(target=producer, args=(q,))
cons = Thread(target=consumer, args=(q,))
prod.start()
cons.start()
prod.join()
cons.join()


enqueued
enqueued
dequeued  79
enqueued
enqueued
dequeued  91
enqueued
dequeued  67
enqueued
enqueued
dequeued  76
enqueued
enqueued
dequeued  3
enqueued
enqueued
dequeued  45
enqueued
enqueued
dequeued  69
enqueued
enqueued
enqueued
dequeued  98
enqueued
enqueued
dequeued  3
enqueued
queue is full, waiting
dequeued  16
enqueued
queue is full, waiting
dequeued enqueued
 31
queue is full, waiting
dequeued enqueued
 37
queue is full, waiting
dequeued enqueued
 84
queue is full, waiting
dequeued  5
enqueued
queue is full, waiting
dequeued  23
enqueued
queue is full, waiting
dequeued  2
enqueued
queue is full, waiting
dequeued  64
enqueued
queue is full, waiting
dequeued  13
enqueued
queue is full, waiting
dequeued  98
enqueued
queue is full, waiting
dequeued  24
enqueued
producer finished
dequeued  6
dequeued  92
dequeued  25
dequeued  98
dequeued  70
dequeued  10
dequeued  4
dequeued  32
dequeued  76
dequeued  80
consumer finished


In [8]:
prod = Thread(target=producer, args=(q,))
prod2 = Thread(target=producer, args=(q,))
cons = Thread(target=consumer, args=(q,))
cons2 = Thread(target=consumer, args=(q,))
prod.start()
cons.start()
prod2.start()
cons2.start()

prod.join()
cons.join()
prod2.join()
cons2.join()

enqueued
enqueued
enqueued
dequeued  8
enqueued
dequeued  1
enqueued
enqueued
enqueued
dequeued  53
enqueued
enqueued
dequeued  42
enqueued
dequeued  60
enqueued
enqueued
dequeued  93
enqueued
enqueued
enqueued
dequeued  65
enqueued
enqueued
dequeued  90
enqueued
queue is full, waiting
dequeued  75
enqueued
queue is full, waiting
queue is full, waiting
dequeued  29
enqueued
dequeued  95
enqueued
queue is full, waiting
queue is full, waiting
dequeued enqueued
 49
dequeued  92
enqueued
queue is full, waiting
queue is full, waiting
dequeued enqueued
 91
dequeued  24
enqueued
queue is full, waiting
queue is full, waiting
dequeued  82
enqueued
dequeued  68
enqueued
queue is full, waiting
dequeued enqueued
 97
queue is full, waiting
dequeued  8
enqueued
queue is full, waiting
dequeued  89
enqueued
queue is full, waiting
queue is full, waiting
dequeued  69
enqueued
dequeued enqueued
 19
queue is full, waiting
dequeued  84
enqueued
queue is full, waiting
queue is full, waiting
dequeued  42
enq

In [11]:
import multiprocessing as mp
import random
import time

class PCQueueMP:
    def __init__(self, capacity=10):
        self.mutex= mp.RLock()
        self.queue = mp.Array('d',capacity)
        self.capacity = capacity
        self.n = 0
        self.notempty = mp.Condition(self.mutex)
        self.notfull = mp.Condition(self.mutex)
    def empty(self):
        with self.mutex:
            return self.n == 0
    def full(self):
        with self.mutex:
            return self.n == self.capacity
    def enqueue(self,item):
        with self.mutex:
            while self.n >= self.capacity:
                print("queue is full, waiting")
                self.notfull.wait()
            
            self.queue[self.n] = item
            self.n += 1
            self.notempty.notify()
            
    def dequeue(self):
        with self.mutex:
            while self.n == 0:
                print("queue is empty, waiting")
                self.notempty.wait()
            
            val = self.queue[0]
            for j in range(0,self.n-1):
                self.queue[j] = self.queue[j+1]
            self.n -= 1
            self.notfull.notify()
            return val
            

def producer(pcq):
    for i in range(30):
        time.sleep(0.15+random.random()*0.15)
        pcq.enqueue(random.randint(0,100))
        print("enqueued")
    print("producer finished")
        
def consumer(pcq):
    for i in range(30):
        time.sleep(0.35+random.random()*0.2)
        print("dequeued ",pcq.dequeue())
    print("consumer finished")
        
q = PCQueueMP()

prod = mp.Process(target=producer, args=(q,))
cons = mp.Process(target=consumer, args=(q,))
prod.start()
cons.start()
prod.join()
cons.join()


enqueued
queue is empty, waiting
enqueuedqueue is empty, waiting

enqueuedqueue is empty, waiting

enqueuedqueue is empty, waiting

enqueuedqueue is empty, waiting

enqueuedqueue is empty, waiting

queue is empty, waitingenqueued

queue is empty, waitingenqueued

enqueuedqueue is empty, waiting

enqueuedqueue is empty, waiting

queue is full, waiting


Process Process-3:
Process Process-4:
Traceback (most recent call last):
Traceback (most recent call last):


KeyboardInterrupt: 

  File "/usr/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_3146958/844642812.py", line 53, in consumer
    print("dequeued ",pcq.dequeue())
  File "/tmp/ipykernel_3146958/844642812.py", line 46, in producer
    pcq.enqueue(random.randint(0,100))
  File "/tmp/ipykernel_3146958/844642812.py", line 33, in dequeue
    self.notempty.wait()
  File "/tmp/ipykernel_3146958/844642812.py", line 23, in enqueue
    self.notfull.wait()
  File "/usr/lib/python3.9/multiprocessing/synchronize.py", line 261, in wait
    return self._wait_semaphore.acquire(True, timeout)
  File "/usr/lib/python3.9/multiprocessing/synch

# multiprocessing.Queue and Queue modules

```from multiprocessing import ..., Queue```

```from threading import ....
    import Queue
```

A synchronized object in shared memory. All blocking/unblocking is already implemented.

In [14]:
from multiprocessing import Queue,Process

q = Queue(10)

def producer(q):
    for i in range(100):
        time.sleep(0.15)
        q.put(i)
def consumer(q):
    for i in range(100):
        item = q.get()
        time.sleep(0.05)
        print(item)

prod=Process(target=producer, args=(q,))
cons=Process(target=consumer, args=(q,))
prod.start()
cons.start()
prod.join()
cons.join()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


# Process/Thread Pools

* `[i1, ... , iN]` items and apply `f()` to all of them in parallel to get `[f(i1), f(i2), ... f(N)]` as a result.
* creating `N` threads/process looks logical but resources are limited. `N == 4` it is ok but if `N == 10000`?.
* Instead create `M` processes and compute in groups of
 `M`.


In [16]:
from multiprocessing import Pool

pool = Pool(8)

def f(i):
    time.sleep(0.2+0.3*random.random())
    return i*i

g = pool.map(f, [i for i in range(100)])
print(g)

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225, 256, 289, 324, 361, 400, 441, 484, 529, 576, 625, 676, 729, 784, 841, 900, 961, 1024, 1089, 1156, 1225, 1296, 1369, 1444, 1521, 1600, 1681, 1764, 1849, 1936, 2025, 2116, 2209, 2304, 2401, 2500, 2601, 2704, 2809, 2916, 3025, 3136, 3249, 3364, 3481, 3600, 3721, 3844, 3969, 4096, 4225, 4356, 4489, 4624, 4761, 4900, 5041, 5184, 5329, 5476, 5625, 5776, 5929, 6084, 6241, 6400, 6561, 6724, 6889, 7056, 7225, 7396, 7569, 7744, 7921, 8100, 8281, 8464, 8649, 8836, 9025, 9216, 9409, 9604, 9801]


Process ForkPoolWorker-19:
Process ForkPoolWorker-23:
Process ForkPoolWorker-22:
Process ForkPoolWorker-26:
Process ForkPoolWorker-24:
Process ForkPoolWorker-21:
Process ForkPoolWorker-20:
Process ForkPoolWorker-25:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.9/mu

Implement your own pool?


# Deadlock and Dining Philosophers

In [17]:
from threading import *
from time import *
from random import *
STARTED = 0
THINKING = 1
HUNGRY = 2
EATING = 3
EXITTED = 4

stmess = "0?-*X"


class Philosopher(Thread):
    def __init__(self,id,forks,states, updated):
        Thread.__init__(self)
        self.id = id
        self.left = forks[0]
        self.right = forks[1]
        self.states = states
        self.states[id] = STARTED
        self.term = False
        self.updated = updated
    def terminate(self):
        self.term = True
    def run(self):
        for i in range(10):
            if self.term:
                break
#            print self.id," is thinking"
            with self.updated:
                self.states[self.id] = THINKING
                self.updated.notify()
        
            sleep(random()*1)
            with self.updated:
                self.states[self.id] = HUNGRY
                self.updated.notify()
            #if self.id % 2 == 0:
            self.left.acquire()
            self.right.acquire()
            #else:
            #    self.right.acquire()
            #    self.left.acquire()
            with self.updated:
                self.states[self.id] = EATING
                self.updated.notify()
#            print self.id," is eating"
            sleep(random()*4)
            self.left.release()
            self.right.release()
        with self.updated:
            self.states[self.id] = EXITTED

print("Enter number of philosopher: ", end='')
n = int(input())

forks = [Lock() for i in range(n)]

phils = []

states = [0 for i in range(n)]

updated = Condition()

for i in range(n):
    phils.append( Philosopher(i,(forks[i],forks[(i+1)%n]),states, updated) )


for phil in phils:
    phil.start() 

while True:
    eflag = True
    for i in range(n):
        if states[i] != EXITTED:
            eflag = False
        print(stmess[states[i]],end='')
    print()
    if eflag:
        break
    try:
        with updated:
            updated.wait()
    except KeyboardInterrupt:
        for phil in phils:
            phil.terminate()

for phil in phils:
    phil.join() 




Enter number of philosopher: 10
??????????
????????*?
????*???*?
????*?*?*?
??*?*?*?*?
??*?*?*-*?
*?*?*?*-*?
*?*?*?*-*-
*?*?*-*-*-
*?*-*-*-*-
*-*-*-*-*-
*-*-?-*-*-
*-*-?*?-*-
*-*-?*?*?-
*-*-?*?*--
?-*-?*?*-*
?-*--*?*-*
?-*--*-*-*
?*?--*-*-*
???--*-*-*
-??--*-*-*
-*?--*-*-*
-*---*-*-*
-*---*-*-?
-*--*?-*-?
-*--*?*?*?
-*--*?*-*?
-*--*?*-*-
-*--*-*-*-
-*--*-*-?-
-*--*-*---
*?--*-*---
*?-*?-*---
*--*?-*---
?--*?-*---
?--*?-*--*
?--*--*--*
---*--*--*
--*?--*--*
--*?--?--*
--*?-*?--*
--*?-*---*
--*--*---*
--*--*--*?
-*?--*--*?
-*---*--*?
-*---*--*-
*?---*--*-
*----*--*-
*---*?--*-
*---*---*-
*--*?---*-
*--*----*-
?--*----*-
?--*----?-
?--*---*?*
---*---*?*
---*---*-*
---*---?-*
---*--*?-*
---*--*--*
--*?--*--*
--*?-*?--*
-*??-*?--*
-*?--*?--*
-*---*?--*
-*---*---*
-*---*--*?
-?---*--*?
*?---*--*?
*----*--*?
*---*?--*?
*---*?--*-
*---*---*-
?---*---*-
?--*?---*-
?--*----*-
---*----*-
---*---*?-
---*---*--
--*?---*--
--*----*--
--*---*?--
-*?---*?--
-*?---*---
-*----*---
*?----*---
*?---*?---


Process ForkPoolWorker-39:
Process ForkPoolWorker-35:
Process ForkPoolWorker-34:
Process ForkPoolWorker-36:
Process ForkPoolWorker-40:
Process ForkPoolWorker-41:
Process ForkPoolWorker-30:
Process ForkPoolWorker-42:
Process ForkPoolWorker-27:
Process ForkPoolWorker-37:
Process ForkPoolWorker-31:
Process ForkPoolWorker-33:
Traceback (most recent call last):
Process ForkPoolWorker-32:
Process ForkPoolWorker-29:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-28:
Process ForkPoolWorker-38:
  File "/usr/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.9/multiprocessing/process.py", line 31

KeyboardInterrupt
  File "/usr/lib/python3.9/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/usr/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
KeyboardInterrupt
  File "/usr/lib/python3.9/multiprocessing/queues.py", line 365, in get
    with self._rlock:
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
  File "/usr/lib/python3.9/multiprocessing/queues.py", line 365, in get
    with self._rlock:
  File "/usr/lib/python3.9/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.9/multiprocessing/queues.py", line 365, in get
    with self._rlock:
KeyboardInterrupt
  File "/usr/lib/python3.9/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/usr/lib/python3.9/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__

# Synchronizing/Watching a Thread/Process

* Have a condition variable for synchronization.
* Send it to Thread/Process
* In the watcher wait for it
* When the model/state changes in thread/process, notify the condition variable.

Call a function in a thread asynchronously (assume there are multiple threads, join() only joins one of them):
```python
def f(x):
    return x*x

def call(c):
    with c[3]:
        c[0] = c[1](c[2])
        c[3].notify()

c = Condition()
# (result, function, input, condition)
result=[None, f, 15, c]
t = Thread(target=call, args=(result))
with c:
    c.wait()
```


In [18]:
from threading import Thread,Condition,Lock
import time

class AsyncCall(Thread):
    def __init__(self,func,args):
        super().__init__()
        self.func = func
        self.args=args
        self.cond=Condition()
        self.ready = False
        self.start()
    def run(self):
        self.value = self.func(self.args)
        with self.cond:
            self.ready = True
            self.cond.notifyAll()
    def wait(self):
        with self.cond:
            while not self.ready:
                self.cond.wait()

def f(x):
    time.sleep(3)
    return x*x

c = AsyncCall(f,10000)
print("I can do usefull stuff here...")
c.wait()
print(c.value)

I can do usefull stuff here...
100000000


# Concurrency Overview 

* Watch race conditions! Use locks/semaphores to protect them
* Watch deadlocks. Be careful when holding a lock and try to acquire another.
* Never make assumptions about timing!. Timing of a thread becoming ready, calling some heavy function. OS/|PL scheduler can behave undeterministically.
* Never busy wait!
* Use monitors and condition variables when you need higher level abstractions of synchronization.
  a monitor queue for producer consumer
* Be careful about if your data is shared or not!
  multiprocessing: not shared, use Value/Array/Queue
  threading: all globals and **object** parameters are shared
* Be careful about Global Interpreter Lock:
   If task is I/O intensive threading should work. but if it has cpu intensive mostly -> no parallelism.
   threading: lightweight, shared variables default, easy to manage but worse parallelism
   multiprocessing: parallel, but more expensive, needs explicit shared variables
* If a process/thread has behavior, implement as a derived class
  ```python
     class myclass(Process):   or class myclass(Thread)
  ```
  call `super().__init__()` in constructor
  override `run()` method.
  if a simple function, just start it.
* If only synchronization is required, your classed can be anything, implement a monitor